In [ ]:
from matplotlib import pyplot as plt
from importlib import reload
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import scipy
import math


In [2]:
class ResidentialLoad:
    
    # Initializer
    def __init__(self):
        self.csv_path = None
        self.dataframe = pd.Dataframe()
        self.isDH = False
        self.isFlex = False
        self.isNew = False
        
    
class HouseNew(ResidentialLoad):
    # Initializer
    def __init__(self):
        self.csv_path = '../data/new_houses.csv'
        self.isNew = True

class HouseOld(ResidentialLoad):
    # Initializer
    def __init__(self):
        self.csv_path = '../data/old_houses.csv'
        
class HouseDH(Residential):
    # Initializer
    def __init__(self):
        
    

In [4]:
    # Class Attributes
load_csv_path = {'house_new':'../data/new_houses.csv', 
                     'house_old':'../data/old_houses.csv', 
                     'house_mix_DH': '../data/mixed_ages_houses_district_heating.csv', 
                     'appart_DH': '../data/new_appartments_district_heating.csv'
                    }

In [5]:
# A function to copy a load profile with stochastic deviation
# from the original. The values varies according to a gaussian 
# distribution, with default and mu = 0, sigma = 0.1. The
# funtion returns a copy of the dataframe with the new column. 

def copy_load_stochastic(dataframe, column_name, sigma=0.1):
    
    min_prob, max_prob = -sigma, sigma
    prob_array = (max_prob - min_prob) * np.random.random_sample(size=dataframe.shape[0]) + min_prob
    new_col_name = column_name + '_stoch_copy'
    dataframe[new_col_name] = dataframe[column_name] + dataframe[column_name].mul(prob_array)
    
    return dataframe

In [8]:
# Subfunction for creating a substation. 
# Triggering just input based on a predefined 
# list of load types.

def input_allocation_dict(load_type_list):
    allocation_dict = dict()
    for load in load_type_list:
        allocation_dict[load] = int(input('Number of '+ load + ': '))
    return allocation_dict

In [7]:
# Returns an updated dataframe object with
# a new profile added according to the input parameters. 

def read_LP(dataframe,load_type, num, path_dict):
    
    num_list = [i for i in range(0,num +1)]         
    path = path_dict[load_type]
    
    if path != None:
        LP_cols = pd.read_csv(path, index_col = 0, 
                              parse_dates = True, 
                              usecols = num_list) # Fix for out of range. Use stochastic copy?
        LP_cols.columns = [load_type + str(i) for i in np.arange(1,len(LP_cols.columns)+1)]
        
        if dataframe.empty:
            dataframe = LP_cols
        else:  #Inner join for date compatability. Needs fixing if changing index structure!!
            dataframe = dataframe.merge(LP_cols, how = 'inner', left_index=True, right_index=True)          
    
    return dataframe

In [9]:
allocation_dict = input_allocation_dict(load_csv_path.keys()) # Trigger input
LP_dataframe = pd.DataFrame()
    
for key, value in allocation_dict.items(): # Performance here? Map was tricky
    LP_dataframe = read_LP(LP_dataframe, key, value, load_csv_path)

Number of house_new: 0


KeyboardInterrupt: 